### Note: Duplicate this notebook so you can run mutliple processes with different api keys concurrently

The following code cell ensures that the underlying `.py` file gets automatically reloaded after you save any changes.  

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd    
from model import initialize_model, call_llm
import time

together_token2 = "sk_WUzz-XTTaJtEAS_SUGy3yrEyr91fhItr1MrnSKK_Xmg" # The api key from your second together account
model_name = "deepseek-ai/DeepSeek-R1"

/Users/jasonchung/Downloads/nlp/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def preprocess(file_path):
    data = pd.read_json(path_or_buf=file_path, lines=True)
    ids = set()
    rows_to_be_dropped = []
    for i in range(len(data)):
        row = data.iloc[i]
        if row.unique_id in ids:
            rows_to_be_dropped.append(i)
        else:
            ids.add(row.unique_id)
    data = data.drop(rows_to_be_dropped)
    return data

In [3]:
data = preprocess("./data/train.jsonl")

In [4]:
first_partition = data[:1365]
second_partition = data[1365:2730]
third_partition = data[2730:4095]
fourth_partition = data[4095:5460]
fifth_partition = data[5460:6825]
sixth_partition = data[6825:]

In [5]:
labels = []
reasonings = []
raw_output = []
ids = []
client, _ = initialize_model("", defined_api_key=together_token2)
last_time = time.time()

# TODO Change len(data) to len(your_partition)
for i in range(len(fifth_partition)):
    if i > 0 and i % 60 == 0:
        print(i)
        current_time = time.time()
        time.sleep(max(0, 63 - (current_time - last_time)))
        last_time = time.time()

    # TODO Change data to your partition
    current_data = fifth_partition.iloc[i]
    ids.append(current_data.unique_id)
    classification, reasoning = call_llm(client, model_name, current_data.string, current_data.sectionName)
    raw_output.append(classification)
    labels.append(classification)
    reasonings.append(reasoning)
    

id='noCykhV-4Yz4kd-928f5f3d0a02f91c' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743419130 model='google/gemma-2-27b-it' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=1858498052518104000, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='```json\n{\n"classification": "background",\n"reasoning": "The text describes previously published findings about Vang protein localization and its role in wing epithelium. This type of information typically belongs in the background section to provide context for the current study\'s results."\n}\n```', tool_calls=[]))] prompt=[] usage=UsageData(prompt_tokens=181, completion_tokens=60, total_tokens=241)
id='noCymTw-4Yz4kd-928f5f4d3e8af91c' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743419133 model='google/gemma-2-27b-it' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=12815437386298167000, finish_r

In [6]:
df = pd.DataFrame(zip(ids, labels, reasonings), columns=["id", "model_classification", "reasoning"])
df.to_csv("fifth_partition_gemma.csv")